# Assignment 1: Web Scraping

## Objective

Data scientists often need to crawl data from websites and turn the crawled data (HTML pages) to structured data (tables). Thus, web scraping is an essential skill that every data scientist should master. In this assignment, you will learn the followings:


* How to use [requests](http://www.python-requests.org/en/master/) to download HTML pages from a website?
* How to select content on a webpage with [lxml](http://lxml.de/)? 

You can either use Spark DataFrame or [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) to do the assignment. In comparison, pandas.DataFrame has richer APIs, but is not good at distributed computing.

## Preliminary

If this is your first time to write a web scraper, you need to learn some basic knowledge of HTML, DOM, and XPath. I found that this is a good resource: [https://data-lessons.github.io](https://data-lessons.github.io/library-webscraping-DEPRECATED/). Please take a look at

* [Selecting content on a web page with XPath
](https://data-lessons.github.io/library-webscraping-DEPRECATED/xpath/)
* [Web scraping using Python: requests and lxml](https://data-lessons.github.io/library-webscraping-DEPRECATED/04-lxml/). 

Please let me know if you find a better resource. I'll share it with the other students.

## Overview

Imagine you are a data scientist working at SFU. One day, you want to analyze CS faculty data and answer two interesting questions:

1. Who are the CS faculty members?
2. What are their research interests?

To do so, the first thing is to figure out what data to collect.

## Task 1: SFU CS Faculty Members

You find that there is a web page in the CS school website, which lists all the faculty members as well as their basic information. 

In Task 1, your job is to write a web scraper to extract the faculty information from this page: [https://www.sfu.ca/computing/people/faculty.html](https://www.sfu.ca/computing/people/faculty.html).




### (a) Crawling Web Page

A web page is essentially a file stored in a remote machine (called web server). You can use [requests](http://www.python-requests.org/en/master/) to open such a file and read data from it. Please complete the following code to download the HTML page and save it as a text file (like [this](./faculty.txt)). 

In [83]:
import requests
# 1. Download the webpage
url = "https://www.sfu.ca/computing/people/faculty.html"
file_to_dump = "data/faculty.txt"
response = requests.get(url)
# 2. Save it as a text file (named faculty.txt)
with open(file_to_dump,'w') as f_open:
    f_open.write(response.text)
    f_open.close()

### (b) Extracting Structured Data

An HTML page follows the Document Object Model (DOM). It models an HTML page as a tree structure wherein each node is an object representing a part of the page. The nodes can be searched and extracted programmatically using XPath. Please complete the following code to transform the above HTML page to a CSV file (like [this](./faculty_table.csv)). 

In [85]:
import lxml.html as lxml_html
import pandas as pd

# 1. Open faculty.txt
with open(file_to_dump,'r') as text:
    content = lxml_html.fromstring(text.read())

# 2. Parse the HTML page as a tree structure
faculty_info = content.xpath("//div[contains(@class,'text')]")

# 3. Extract related content from the tree using XPath
info_list = []
for els in faculty_info:
  dc = {}
  name = els.xpath("./h4/text()")
  if name:
        dc['name']=name[0].split(',')[0].strip()
        dc['designation']=name[0].split(',')[1].strip()
        dc['area'] = els.xpath("./p/text()")[0].strip()
        urls = els.xpath(".//a/@href")
        if len(urls)>=2:
            dc['profile_page'] = urls[-2].strip()
            dc['home_page'] = urls[-1].strip()
        elif len(urls)==1:
            dc['profile_page'] = urls[0].strip()
            dc['home_page'] = ''
        else:
            dc['profile_page'] = ''
            dc['home_page'] = ''
        info_list.append(dc)
        #print(dc)
# 4. Save the extracted content as an csv file (named faculty_table.csv)
df = pd.DataFrame(info_list)
header = ['name','designation','area','profile_page','home_page']
df.to_csv("data/faculty_table.csv",columns=header)

## Task 2: Research Interests

Suppose you want to know the research interests of each faculty. However, the above crawled web page does not contain such information. 

### (a) Crawling Web Page

You notice that such information can be found on the profile page of each faculty. For example, you can find the research interests of Dr. Jiannan Wang from [http://www.sfu.ca/computing/people/faculty/jiannanwang.html](http://www.sfu.ca/computing/people/faculty/jiannanwang.html). 


Please complete the following code to download the profile pages and save them as text files. There are 60 faculties, so you need to download 60 web pages in total. 

In [103]:
import requests

# 1. Download the profile pages of 60 faculties
header = ['name','designation','area','profile_page','home_page']
faculty_df = pd.read_csv('data/faculty_table.csv',usecols=header)
base_url = "https://www.sfu.ca/"
for i,rows in faculty_df.iterrows():
    if(rows['profile_page']!=''):
        html_resp =  requests.get(base_url + rows['profile_page'])
        fname = "data/faculty_profile_html/"+rows['name'].lower()
        # 2. Save each page as a text file
        with open(fname,'w',encoding='utf-8',errors=None) as f_html:
            f_html.write(html_resp.text)
            f_html.close()

### (b) Extracting Structured Data

Please complete the following code to extract the research interests of each faculty, and generate a file like [this](./faculty_more_table.csv). 

In [130]:
import lxml.html as lxml_html
import os

file_location = "data/faculty_profile_html/"
research_table = []
for files in os.listdir(file_location):
    # 1. Open each text file and parse it as a tree structure
    with open(file_location+files,'r',encoding='utf-8') as html_text:
        # 2. Extract the research interests from each tree using XPath
        content = lxml_html.fromstring(html_text.read())
        #res_interests = content.xpath("//div[contains(@class,'text parbase section')]//div[contains(@class,'listed')]/text()")
        res_interests = content.xpath('//div[contains(@class,"listed")]/h2[contains(.,"Research interests")]')
        dc = {"faculty_name": files,'research_interests':""}
        for els in res_interests:
               dc['research_interests'] = "|".join(els.xpath("..//li/text()"))
        research_table.append(dc)
df = pd.DataFrame(research_table) 
df.to_csv("data/faculty_more_table.csv",columns=['faculty_name','research_interests'])
# 3. Add the extracted content to faculty_table.csv
# 4. Generate a new CSV file, named faculty_more_table.csv

## Submission

Complete the code in this [notebook](A1.ipynb), and submit it to the CourSys activity `Assignment 1`.